In [ ]:
import pandas as pd
import numpy as np
import os
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D,Dropout
from tensorflow.keras.applications.mobilenet import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

In [ ]:

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

In [ ]:
downloaded = drive.CreateFile({'id': '1Jqg5zazL3f4pEi2xBeIvLpfo01GCQxMY'})
downloaded.GetContentFile('cnndh.zip')

In [ ]:
!unzip cnndh.zip

In [ ]:
base_model= MobileNet(weights='imagenet',include_top=False)

In [ ]:
base_model.summary()

In [ ]:
CLASSES = 3
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D(name='avg_pool')(x)
x = tf.keras.layers.Dropout(0.4)(x)
predictions =tf.keras.layers.Dense(3, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
for layer in base_model.layers:
    layer.trainable = False
      
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
WIDTH = 299
HEIGHT = 299
BATCH_SIZE = 32
TRAIN_DIR=r'Dog_Human/train'
TEST_DIR=r'Dog_Human/test'
# data prep
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(HEIGHT, WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='categorical')
    
validation_generator = validation_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(HEIGHT, WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical')
print(train_generator.class_indices)
print(validation_generator.class_indices)

Found 19810 images belonging to 3 classes.
Found 3504 images belonging to 3 classes.
{'dog': 0, 'human': 1, 'nothd': 2}
{'dog': 0, 'human': 1, 'nothd': 2}


In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
EPOCHS = 2
BATCH_SIZE = 32
STEPS_PER_EPOCH = 320
VALIDATION_STEPS=64
model.fit_generator(
    train_generator,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=validation_generator,
    validation_steps=VALIDATION_STEPS
    )

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/2
320/320 [==============================] - 296s 800ms/step - loss: 0.2915 - accuracy: 0.8913 - val_loss: 0.0906 - val_accuracy: 0.9658
Epoch 2/2
320/320 [==============================] - 254s 794ms/step - loss: 0.0463 - accuracy: 0.9863 - val_loss: 0.0521 - val_accuracy: 0.9814


In [ ]:
from tensorflow.python.keras.preprocessing import image

In [ ]:
img = image.load_img(r'Dog_Human/test/dog/1003.jpg', target_size=(299,299))
x = image.img_to_array(img)
x = preprocess_input(x)
x = np.expand_dims(x, axis=0)
pred = model.predict(x)[0]
print(pred)
y_classes = pred.argmax(axis=-1)
y_classes

In [ ]:
img = image.load_img(r'Dog_Human/test/human/Sharon_Osbourne_0003.jpg', target_size=(299,299))
x = image.img_to_array(img)
x = preprocess_input(x)
x = np.expand_dims(x, axis=0)
pred = model.predict(x)[0]
print(pred)
y_classes = np.argmax(pred)
y_classes

In [ ]:
!pip install keras
!pip install keras.models


In [ ]:

from keras.models import model_from_json
model_json = model.to_json()

with open("cnn.json", "w") as json_file:
    json_file.write(model_json)
    
model.save_weights("cnn.h5")

In [ ]:
train_generator.class_indices

{'dog': 0, 'human': 1, 'nothd': 2}